In [2]:
import numpy as np

In [35]:

class activation_function:
    def pass_values(self,input_values):
        pass
    def get_gradient(self,gradient):
        pass

        
class layer:
    def forward(input_values):
        pass
    def get_running_gradient(self,gradient):
        pass
    def get_parameter_gradients(self,gradient,X):
        pass
    def get_parameters():
        pass
        
class optimizer:
    def update_parameter(parameter):
        pass

class cost_function:
    def get_cost():
        pass
    def get_gradient():
        pass

In [36]:
class relu(activation_function):
    def pass_value(input_values):
        return max(input_values,0)
    def get_gradient(gradient):
        return max(gradient,0)
        
        

class sigmoid(activation_function):
    def pass_value(input_values):
        return 1 / (1 + np.exp(-input_values))
    def get_gradient(gradient):
        return gradient * (1- gradient)



        

In [37]:
class SGD(optimizer):
    def __init__(self,alpha):
        self.alpha=alpha
    def update_parameters(self,gradient,parameters):
        if not parameters:
            return
            
        for index,parameter in enumerate(parameters):
            parameter-= gradient[index] * self.alpha


In [38]:
class MSE(cost_function):
    def get_cost(output_values, expected_values):
        return 0.5*np.mean(np.square(expected_values-output_values))
    def get_gradient(output_values, expected_values):
        return output_values.shape[0]**(-1) * (expected_values-output_values)
        
    

In [39]:


def get_activation_functions():
    return {
        "relu":relu,
        "leaky_relu":leaky_relu,
        "sigmoid":sigmoid,
        "tanh":tanh,
    }

def get_cost_functions():
    return {
        "MSE":MSE,
    }

def get_optimizer():
    return {
        "SGD":SGD,
    } 

    

In [44]:
def initialize_weights(shape, activation_function):
    if shape>2:
        fan_in = shape[0] * shape[2] * shape[3]
        fan_out = shape[1] * shape[2] * shape[3]        

    else:
        fan_in=shape[0]
        fan_out=shape[1]

    match activation_function:
        case "sigmoid" | "tanh":
            weights = sqrt(2 / (fan_in + fan_out)) *np.random.randn(*shape)
        case "relu" | "leaky_relu":
            limit = sqrt(6 / fan_in)
            weights = np.random.uniform(-limit,limit, shape)

    return weights
        
    

In [45]:
class linear(layer):
    def __init__(self,input_size,output_size,activation_function):
        self.activation_function=get_activation_functions()[activation_function.lower()]
        self.weights=initialize_weights((input_size,output_size),activation_function.lower())
        self.bias=np.zeros(output_dim)

    def forward(self,input_values):
        return activation_function.pass_value(input_values @ self.weights + self.bias)

    def get_running_gradient(self,gradient):
        gradient=gradient * activation_function.get_gradient()
        self.dX=gradient @ W
        return dX

    def get_parameter_gradients(self,gradient, X):
        self.dW= X.shape[0]**(-1) *( gradient.T @ X )
        self.db= ( gradient.mean(axis=0)) 
        return [self.dW,self.db]

    def get_parameters():
        return [self.weights,self.bias]


In [46]:
    

def conv_pad(kernels):
    pad_dim_kernel=((feature_map.shape[0]-1,feature_map.shape[0]-1),
                    (feature_map.shape[1]-1,feature_map.shape[1]-1))
    
    

    padded_kernel = np.pad(feature_map, pad_width= pad_dim_kernel,
                           mode='constant', constant_values=0)
    return padded_kernel

def valid_convolution_cnn2d(kernels,feature_map,stride,padding=False):
    
    padded_kernel=conv_pad(kernels)
    padded_feature_map=conv_pad(feature_map)

    tranformed_kernels=np.fft.fft2(padded_kernel, axes=(-2,-1))
    tranformed_feature_map=np.fft.fft2(padded_feature_map, axes=(-2,-1))

    transformed_kernels=transformed_kernels[np.newaxis,:,:,:,:]
    transformed_feature_map= transformed_feature_map[:,np.newaxis,:,:,:]
    
    product_result=transformed_kernels*transformed_feature_map

    result=np.fft.ifft2(product_result, axes=(-2,-1))

    if not padding:
        
        crop=(kernels.shape[2] - 1) // 2
        result=result[:,:,:,crop:-crop,crop:-crop]
    
    result=result[:,:,:,::stride,::stride]
    
    return result


In [47]:

class convolutional_2d(layer):
    def __init__(self,input_channel_size, output_channel_size, kernel_size, padding, activation_function):
        self.activation_function=get_activation_functions()[activation_function.lower()]
        self.kernels=initialize_weights((input_channel_size,output_channel_size,kernel_size),activation_function.lower())
        self.bias=np.zeros(output_dim)
        self.padding=self.padding
    def forward(self,feature_map):
        return valid_convolution_cnn2d(self.kernels,feature_map,self.padding)
    
    def get_running_gradient(self,gradient):
        
        dX= valid_convolution_cnn2d(gradient,self.kernels[:,:,::-1,::-1],self.padding)
        return dX

    def get_parameter_gradients(self,gradient, X):
        
        if self.padding:
            X_modified= conv_pad(X)
        else:
            X_modified=X
        
        self.dK= valid_convolution_cnn2d(gradient, X_modified ,False)
        self.db= np.sum(gradient, axis=(0,2,3))/ gradient.shape[0] 
        return [self.dK,self.db]

    def get_parameters():
        return [self.kernels,self.bias]
        
        
class convolutional_3d(layer):
    def __init__(kernel_dimensions, stride, bias, activation_function):
        pass
        
class flatten(layer):
    def forward(self,X):
        self.shape=x.shape
        return X.reshape(x.shape[0], -1)
    
    def get_running_gradient(self,gradient):
        return gradient.reshape(self.shape)
        
    def get_parameters():
        return []
    def get_parameter_gradients():
        return []
        
        
class max_pool(layer):
    def __init__(self):
        pass
        
    def get_parameters():
        return []
    def get_parameter_gradients():
        return []
        

class avg_pool2d(layer):
    def __init__(self):
        pass
        
    def get_parameters():
        return []
    def get_parameter_gradients():
        return []


class max_pool2d(layer):
    def __init__(self):
        pass
        
    def get_parameters():
        return []
    def get_parameter_gradients():
        return []
             
class avg_pool3d(layer):
    def __init__(self):
        pass
        
    def get_parameters():
        return []
    def get_parameter_gradients():
        return []

class max_pool3d(layer):
    def __init__(self):
        pass    
        
    def get_parameters():
        return []
    def get_parameter_gradients():
        return []


In [48]:
def model():
    def __init__(self,layers,cost_function,optimizer, batch_size, X, Y):
        self.layers=layers
        self.optimizer=optimizer
        
        self.layer_values=[np.zeros(batch_size, X.shape[1:])]
        
        self.cost_function=get_cost_function()[cost_function]
        self.Y=Y
        self.X=X
        self.number_of_batches=  ( X.shape[0] - (X.shape[0]%batch_size) ) / batch_size
        self.costs=np.zeros(number_of_batches)
        self.batch_size=batch_size
        self.running_avg=self.batch_size*(-1)

    def train(self):
        for batch in range(self.number_of_batches):
            batch_range= splice(batch*self.batch_size,batch*self.batch_size+self.batch_size)
            self.forward(batch_range)
            self.backward(batch_range)
            
    def save(self):
        pass
    
    def forward(self,batch_range=slice(None)):

        
        labels= self.Y[batch_range]
        self.layer_values[0]=X[batch_range]
        
        for layer_number, layer in enumerate(self.layers):
            layer_values.append(layer.forward(self.layer_values[layer_number]))

        self.costs= self.cost_function.get_cost(self.layer_values[-1], labels)

            
       
    def backward(self, batch_range):
        
        labels= self.Y[batch_range]
        running_gradient=self.cost_function.get_gradient(self.layer_values[-1], labels)

        for layer_number in range(-1,-len(self.layers),-1):
                        
            parameter_update_gradient = self.layers[layer_number].get_parameter_gradients(
                running_gradient, self.layer_values[layer_number-1])
                
            running_gradient= self.layers[layer_number].get_running_gradient(running_gradient)

            self.optimizer.update_parameter(parameter_update_gradient, self.layers[layer_number].get_parameters())
                
            
            

    
        
        